In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('/content/data.csv',delimiter=',',encoding='latin-1')
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [4]:
df.shape

(5842, 2)

In [5]:
pd.set_option('display.max_colwidth', None)
messages = df[['Sentence','Sentiment']]
messages.columns = ["text", "label"]
messages.head()

,text,label
0,"The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .",positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real possibility",negative
2,"For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .",positive
3,"According to the Finnish-Russian Chamber of Commerce , all the major construction companies of Finland are operating in Russia .",neutral
4,"The Swedish buyout firm has sold its remaining 22.4 percent stake , almost eighteen months after taking the company public in Finland .",neutral


In [6]:
import gensim
messages['text_clean_gensim'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x,deacc=True))
messages.head()

,text,label,text_clean_gensim
0,"The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .",positive,"[the, geosolutions, technology, will, leverage, benefon, gps, solutions, by, providing, location, based, search, technology, communities, platform, location, relevant, multimedia, content, and, new, and, powerful, commercial, model]"
1,"$ESI on lows, down $1.50 to $2.50 BK a real possibility",negative,"[esi, on, lows, down, to, bk, real, possibility]"
2,"For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .",positive,"[for, the, last, quarter, of, componenta, net, sales, doubled, to, eur, from, eur, for, the, same, period, year, earlier, while, it, moved, to, zero, pre, tax, profit, from, pre, tax, loss, of, eur]"
3,"According to the Finnish-Russian Chamber of Commerce , all the major construction companies of Finland are operating in Russia .",neutral,"[according, to, the, finnish, russian, chamber, of, commerce, all, the, major, construction, companies, of, finland, are, operating, in, russia]"
4,"The Swedish buyout firm has sold its remaining 22.4 percent stake , almost eighteen months after taking the company public in Finland .",neutral,"[the, swedish, buyout, firm, has, sold, its, remaining, percent, stake, almost, eighteen, months, after, taking, the, company, public, in, finland]"


In [7]:
messages['label']=messages['label'].map({'negative':0,'positive':1,'neutral':2})

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (messages['text_clean_gensim'], messages['label'] , test_size=0.2,random_state=0)

# Traning W2V>>>>>> Word to Vector

In [22]:
w2v_model = gensim.models.Word2Vec(X_train, min_count=1)

In [23]:
words = w2v_model.wv.index2word  #word count

In [24]:
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_test])

<ipython-input-24-5540b7745640>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_train])
<ipython-input-24-5540b7745640>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_test])


In [25]:
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Random Forest classifier

In [57]:
from sklearn.ensemble import RandomForestClassifier
  
clf = RandomForestClassifier(n_estimators = 100) 
clf.fit(X_train_vect_avg, y_train)

RandomForestClassifier()

In [58]:
from sklearn import metrics
y_pred = clf.predict(X_train_vect_avg) 
metrics.accuracy_score(y_pred, y_train)

0.9300235394821313

In [59]:
from sklearn import metrics
y_pred = clf.predict(X_test_vect_avg) 
metrics.accuracy_score(y_pred, y_test)

0.5329341317365269

**train accuracy=93......test accuracy=54**

# Naive Bayes

In [29]:
from sklearn.naive_bayes import GaussianNB
classifier=GaussianNB()
classifier.fit(X_train_vect_avg, y_train)


GaussianNB()

In [31]:
from sklearn import metrics
y_pred=classifier.predict(X_train_vect_avg)
metrics.accuracy_score(y_pred,y_train)

0.5245024609458592

In [69]:
from sklearn import metrics
y_pred=classifier.predict(X_test_vect_avg)
metrics.accuracy_score(y_pred,y_test)

0.5397775876817793

**train accuracy= 52.4..........test accuracy=54**